<a href="https://colab.research.google.com/github/isikaykarakus/ForeoAIINternship/blob/main/foreow2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# -*- coding: utf-8 -*-
# Multilingual PoC: EN/ES/PL/TR + Gemma 270M (with safe fallbacks) + style control

!pip -q install sentence-transformers faiss-cpu transformers pandas

import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.9 MB/s eta 0:00:00


In [ ]:
# -----------------------------
# 1) Tiny multilingual dataset
# -----------------------------
rows = [
    # English
    {"lang":"en","phrase":"spill the tea","meaning":"share gossip or a secret","usage":"She spilled the tea about the new launch.","source_url":"https://example.com"},
    {"lang":"en","phrase":"low-key","meaning":"subtly; a little bit; not openly","usage":"I’m low-key excited about this collab.","source_url":"https://example.com"},
    # Spanish
    {"lang":"es","phrase":"estar en las nubes","meaning":"estar distraído; no prestar atención","usage":"En clase siempre está en las nubes.","source_url":"https://example.com"},
    {"lang":"es","phrase":"ponerse las pilas","meaning":"empezar a esforzarse; ponerse activo","usage":"Tenemos que ponernos las pilas antes del lanzamiento.","source_url":"https://example.com"},
    # Polish
    {"lang":"pl","phrase":"mieć muchy w nosie","meaning":"być markotnym; mieć zły humor","usage":"Dziś ma muchy w nosie i nie chce rozmawiać.","source_url":"https://example.com"},
    {"lang":"pl","phrase":"nie być w sosie","meaning":"mieć gorszy dzień; być nie w nastroju","usage":"Szef jest dziś nie w sosie.","source_url":"https://example.com"},
    # Turkish
    {"lang":"tr","phrase":"kafayı yemek","meaning":"çok sinirlenmek ya da aklını kaçıracak gibi olmak","usage":"Sunum bozulunca az daha kafayı yiyordum.","source_url":"https://example.com"},
    {"lang":"tr","phrase":"gaza gelmek","meaning":"kolayca coşup harekete geçmek; kışkırtılmak","usage":"Arkadaşları söyleyince hemen gaza geldi.","source_url":"https://example.com"},
]
df = pd.DataFrame(rows)
display(df)


,lang,phrase,meaning,usage,source_url
0,en,spill the tea,share gossip or a secret,She spilled the tea about the new launch.,https://example.com
1,en,low-key,subtly; a little bit; not openly,I’m low-key excited about this collab.,https://example.com
2,es,estar en las nubes,estar distraído; no prestar atención,En clase siempre está en las nubes.,https://example.com
3,es,ponerse las pilas,empezar a esforzarse; ponerse activo,Tenemos que ponernos las pilas antes del lanza...,https://example.com
4,pl,mieć muchy w nosie,być markotnym; mieć zły humor,Dziś ma muchy w nosie i nie chce rozmawiać.,https://example.com
5,pl,nie być w sosie,mieć gorszy dzień; być nie w nastroju,Szef jest dziś nie w sosie.,https://example.com
6,tr,kafayı yemek,çok sinirlenmek ya da aklını kaçıracak gibi olmak,Sunum bozulunca az daha kafayı yiyordum.,https://example.com
7,tr,gaza gelmek,kolayca coşup harekete geçmek; kışkırtılmak,Arkadaşları söyleyince hemen gaza geldi.,https://example.com


In [ ]:

# ------------------------------------------
# 2) Multilingual embeddings + FAISS index
# ------------------------------------------
# Use a compact multilingual model so all 4 languages live in the same space
EMB_ID = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
emb_model = SentenceTransformer(EMB_ID)

# What we embed for retrieval (language-agnostic)
df["blob"] = df["phrase"] + " — " + df["meaning"] + " — " + df["usage"]
embeddings = emb_model.encode(df["blob"].tolist(), normalize_embeddings=True)

# Cosine via inner product on normalized vectors
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:


# -----------------------------------------
# 3) Generator: prefer Gemma 270M, fallback
# -----------------------------------------
# Some Gemma variants require accepting license / HF login.
# We'll attempt Gemma-270M first; if that fails, fall back to a tiny open model.
MODEL_CANDIDATES = [
    "google/gemma-3-270m",     # preferred (if available to you)
    "google/gemma-2-2b-it",    # small-ish instruct fallback
    "google/flan-t5-small"     # tiny fallback that always works
]

loaded_id = None
gen = None
for mid in MODEL_CANDIDATES:
    try:
        tok = AutoTokenizer.from_pretrained(mid)
        if "t5" in mid.lower():
            # text2text style
            gen = pipeline("text2text-generation", model=mid, tokenizer=tok)
        else:
            # causal LM style
            lm = AutoModelForCausalLM.from_pretrained(mid)
            gen = pipeline("text-generation", model=lm, tokenizer=tok)
        loaded_id = mid
        break
    except Exception as e:
        print(f"[warn] Could not load {mid}: {e}")

print(f"[info] Loaded generator: {loaded_id}")


[warn] Could not load google/gemma-3-270m: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-3-270m.
403 Client Error. (Request ID: Root=1-68de4b1a-4697811829d8c6f70b8d44c7;c3b05bdc-2d01-4794-94ea-b5a0b5dde714)

Cannot access gated repo for url https://huggingface.co/google/gemma-3-270m/resolve/main/config.json.
Access to model google/gemma-3-270m is restricted and you are not in the authorized list. Visit https://huggingface.co/google/gemma-3-270m to ask for access.
[warn] Could not load google/gemma-2-2b-it: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2-2b-it.
403 Client Error. (Request ID: Root=1-68de4b1b-1ed093053181b7cf1579fe10;0f8794a0-0bc0-48bd-909c-397acd9ae4ee)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-2b-it/resolve/main/config.json.
Access to model google/gemma-2-2b-it is restricted and you are not in the authorized list. Vi

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


[info] Loaded generator: google/flan-t5-small


In [ ]:

# -------------------------------
# 4) Retrieval & explanation
# -------------------------------
STYLES = {
    "learner": "Explain simply for language learners. Avoid slang in the explanation and include ONE short example.",
    "casual":  "Use a casual, friendly tone and keep it short.",
    "formal":  "Use a clear, formal, brand-safe tone suitable for documentation."
}

def search(query: str, k: int = 3):
    qv = emb_model.encode([query], normalize_embeddings=True)
    D, I = index.search(qv, k)
    hits = df.iloc[I[0]].copy()
    hits["score"] = [float(s) for s in D[0]]
    return hits

def _gen_text(prompt: str):
    # Works for both text2text and causal pipelines
    if "t5" in (loaded_id or "").lower():
        return gen(prompt, max_new_tokens=140)[0]["generated_text"]
    else:
        return gen(prompt, max_new_tokens=140, do_sample=False)[0]["generated_text"]

def explain(query: str, style: str = "learner", k: int = 3):
    hits = search(query, k)
    context = "\n".join([f"- [{r.lang}] {r.phrase}: {r.meaning} (e.g., {r.usage})" for _, r in hits.iterrows()])
    style_instr = STYLES.get(style, STYLES["learner"])

    prompt = (
        f"Explain the expression '{query}'. {style_instr}\n"
        f"Use the retrieved examples below as context and mention the language code in examples.\n"
        f"Retrieved examples:\n{context}\n\n"
        f"Answer:"
    )
    out = _gen_text(prompt)
    return hits[["lang","phrase","meaning","usage","source_url","score"]], out


In [ ]:

# -----------------------------------
# 5) Run one example per language
# -----------------------------------
queries = ["spill the tea", "estar en las nubes", "mieć muchy w nosie", "kafayı yemek"]
for q in queries:
    table, answer = explain(q, style="learner", k=3)
    display(table)
    print(f"\n--- {q} | learner ---\n{answer}\n")

# Show style customisation on one term
_, ans_casual = explain("spill the tea", style="casual", k=3)
print("\n=== STYLE: CASUAL ===\n", ans_casual)
_, ans_formal = explain("spill the tea", style="formal", k=3)
print("\n=== STYLE: FORMAL ===\n", ans_formal)


,lang,phrase,meaning,usage,source_url,score
0,en,spill the tea,share gossip or a secret,She spilled the tea about the new launch.,https://example.com,0.664074
7,tr,gaza gelmek,kolayca coşup harekete geçmek; kışkırtılmak,Arkadaşları söyleyince hemen gaza geldi.,https://example.com,0.300972
5,pl,nie być w sosie,mieć gorszy dzień; być nie w nastroju,Szef jest dziś nie w sosie.,https://example.com,0.298178



--- spill the tea | learner ---
[en] spill the tea: share gossip or a secret (e.g., She spilled the tea about the new launch.) - [tr] gaza gelmek: kolayca coşup harekete geçmek; kşkrtlmak (e.g., Arkadaşlar söyleyince hemen gaza geldi.)



,lang,phrase,meaning,usage,source_url,score
2,es,estar en las nubes,estar distraído; no prestar atención,En clase siempre está en las nubes.,https://example.com,0.701934
5,pl,nie być w sosie,mieć gorszy dzień; być nie w nastroju,Szef jest dziś nie w sosie.,https://example.com,0.420594
4,pl,mieć muchy w nosie,być markotnym; mieć zły humor,Dziś ma muchy w nosie i nie chce rozmawiać.,https://example.com,0.403082



--- estar en las nubes | learner ---
[es] estar en las nubes: estar distrado; no prestar atención (e.g., En clase siempre está en las nubes.) - [pl] nie by w sosie: mie gorszy dzie; by nie w nastroju (e.g., Szef jest dzi nie w sosie.) - [pl] mie muchy w nosie: by markotnym; mie zy



,lang,phrase,meaning,usage,source_url,score
4,pl,mieć muchy w nosie,być markotnym; mieć zły humor,Dziś ma muchy w nosie i nie chce rozmawiać.,https://example.com,0.801063
6,tr,kafayı yemek,çok sinirlenmek ya da aklını kaçıracak gibi olmak,Sunum bozulunca az daha kafayı yiyordum.,https://example.com,0.411555
2,es,estar en las nubes,estar distraído; no prestar atención,En clase siempre está en las nubes.,https://example.com,0.334462



--- mieć muchy w nosie | learner ---
[pl] mie muchy w nosie: by markotnym; mie zy humor (e.g., Dzi ma muchy w nosie i nie chce rozmawia.) - [tr] kafay yemek: çok sinirlenmek ya da akln kaçracak gibi olmak (e.g., Sunum bozulunca az daha kafay y



,lang,phrase,meaning,usage,source_url,score
6,tr,kafayı yemek,çok sinirlenmek ya da aklını kaçıracak gibi olmak,Sunum bozulunca az daha kafayı yiyordum.,https://example.com,0.713488
4,pl,mieć muchy w nosie,być markotnym; mieć zły humor,Dziś ma muchy w nosie i nie chce rozmawiać.,https://example.com,0.441232
5,pl,nie być w sosie,mieć gorszy dzień; być nie w nastroju,Szef jest dziś nie w sosie.,https://example.com,0.403120



--- kafayı yemek | learner ---
[tr] kafay yemek: çok sinirlenmek ya da akln kaçracak gibi olmak (e.g., Sunum bozulunca az daha kafay yiyordum.) - [pl] mie muchy w nosie: by markotnym; mie zy humor (e.g., Dzi ma muchy w nosie i nie chce rozm


=== STYLE: CASUAL ===
 [en] spill the tea: share gossip or a secret (e.g., She spilled the tea about the new launch.) - [tr] gaza gelmek: kolayca coşup harekete geçmek; krtlmak (e.g., Arkadaşlar söyleyince hemen gaza geldi.) - [pl]

=== STYLE: FORMAL ===
 [en] spill the tea: share gossip or a secret (e.g., She spilled the tea about the new launch.) - [en] spill the tea: share gossip or a secret (e.g., She spilled the tea about the new launch.) - [en] spill the tea: share gossip or a secret (e.g., She spilled the tea about the new launch.) - [en] spill the tea: share gossip or a secret (e.g., She spilled the tea about the new launch.) - [en] spill the tea: share
